# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from random import sample

import re

import xgboost as xgb

from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV

from timeit import default_timer as timer

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [5]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9707,HOBBIES_2_145_CA_4_validation,HOBBIES_2_145,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9708,HOBBIES_2_146_CA_4_validation,HOBBIES_2_146,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9709,HOBBIES_2_147_CA_4_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9710,HOBBIES_2_148_CA_4_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Hobbies'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [7]:
len(sales_intermittent)

869

In [15]:
len(sales_lumpy)

231

In [16]:
len(sales_erratic)

99

In [ ]:
len(sales_smooth)

3594

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [8]:
sales_erratic

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
73,HOBBIES_1_076_CA_1_validation,HOBBIES_1_076,HOBBIES_1,HOBBIES,CA_1,CA,0,1,2,3,...,0,2,1,0,0,0,0,1,1,0
77,HOBBIES_1_080_CA_1_validation,HOBBIES_1_080,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,12,2,0,0,7
88,HOBBIES_1_091_CA_1_validation,HOBBIES_1_091,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,7,1,3,2,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9292,HOBBIES_1_150_CA_4_validation,HOBBIES_1_150,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
9298,HOBBIES_1_156_CA_4_validation,HOBBIES_1_156,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,0,2,1,0,0,0,0,0,0,0
9342,HOBBIES_1_202_CA_4_validation,HOBBIES_1_202,HOBBIES_1,HOBBIES,CA_4,CA,2,1,5,3,...,0,0,0,0,0,0,0,0,0,0
9362,HOBBIES_1_223_CA_4_validation,HOBBIES_1_223,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,2,0,0,0,1,0,0,0,1,0


# User-defined functions to calculate Metrics

In [9]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Format sales data and then merge with calendar data

In [10]:
sales_pattern = sales_erratic.copy()

In [11]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)

dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()


In [17]:
lags = list(range(1, 25))
moving_averages = list(range(1, 25)) 
stds = list(range(2, 25))
maximums = list(range(1, 25))
minimums = list(range(1, 25))
totals = list(range(1, 25))
num_days_zeros = list(range(1, 25))

correlation_threshold = 0.8

end_train_week = 11610

num_id_sample_for_tuning = 50

In [18]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [19]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [20]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [21]:
len(sample_list_filtered)

38

In [ ]:
start = timer()

df_output_hyper_params = pd.DataFrame()
    
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    dataframe_product = dataframe_weekly[['wm_yr_wk', product]]

    # Create Lag variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (t-{lag})': dataframe_product[product].shift(lag)
        for lag in lags
    })

    # Create Rolling Moving Average variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
        for moving_average in moving_averages
    })

    # Create Rolling Standard Deviation variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
        for std in stds
    })

    # Create Rolling Maximum variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
        for maximum in maximums
    })

    # Create Rolling Minimum variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
        for minimum in minimums
    })

    # Create Rolling Total variables 
    dataframe_product = dataframe_product.assign(**{
        f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
        for total in totals
    })

    # Create Zero Count variables
    count_func = lambda x: (x==0).sum()

    dataframe_product = dataframe_product.assign(**{
        f'{product} (zero_count_in_{num_days_zero})': dataframe_product[product].shift(1).rolling(num_days_zero).apply(count_func)
        for num_days_zero in num_days_zeros
    })

    dataframe_product.dropna(inplace=True)
    dataframe_product.set_index('wm_yr_wk', inplace=True)

    train_data = dataframe_product[dataframe_product.index <= end_train_week]

    corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
    high_corr_var=np.where(corr_matrix > correlation_threshold)
    high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

    train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)

    X_train = train_data.drop(product, axis=1)
    y_train = train_data[product]  

    tscv = TimeSeriesSplit(n_splits=3)

    params = { 'max_depth': [3, 5, 7, 10],
              'learning_rate': [0.01, 0.1, 0.2, 0.3],
              'subsample': np.arange(0.5, 1.0, 0.1),
              'colsample_bytree': np.arange(0.5, 1.0, 0.1),
              'colsample_bylevel': np.arange(0.5, 1.0, 0.1),
              'n_estimators': [30, 50, 100]}

    xgbr = xgb.XGBRegressor(seed = 20)

    regressor = RandomizedSearchCV(estimator=xgbr,
                                  param_distributions=params,
                                  cv = tscv, 
                                  scoring='neg_mean_absolute_percentage_error',
                                  n_iter=300,
                                  verbose=1)
    regressor.fit(X_train, y_train)
    best_params_temp = regressor.best_params_

    df_output_temp = pd.DataFrame.from_dict(best_params_temp, orient='index').T
    df_output_temp['Product'] = product

    df_output_hyper_params = df_output_hyper_params.append(df_output_temp, ignore_index=True)

    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Streaming output truncated to the last 5000 lines.
[14:10:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:10:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:10:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:10:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:10:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:10:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:10:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:10:27] WARNING: /workspace/src/objective/regression_obj.

In [ ]:
df_output_hyper_params

,subsample,n_estimators,max_depth,learning_rate,colsample_bytree,colsample_bylevel,Product
0,0.8,30.0,10.0,0.01,0.9,0.8,FOODS_2_031_CA_3
1,0.5,50.0,5.0,0.10,0.6,0.5,FOODS_3_332_CA_3
2,0.9,30.0,10.0,0.01,0.8,0.6,FOODS_2_266_CA_3
3,0.6,30.0,10.0,0.01,0.6,0.7,FOODS_2_378_CA_3
4,0.9,30.0,7.0,0.01,0.9,0.5,FOODS_3_788_CA_1
...,...,...,...,...,...,...,...
75,0.6,30.0,7.0,0.01,0.6,0.8,FOODS_3_271_CA_3
76,0.5,50.0,10.0,0.20,0.6,0.5,FOODS_3_437_CA_2
77,0.5,30.0,5.0,0.01,0.9,0.7,FOODS_1_171_CA_2
78,0.6,30.0,10.0,0.01,0.9,0.7,FOODS_3_070_CA_2


In [ ]:
df_output_hyper_params.to_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/XGBoost (Completed Pipeline)/Foods/Erratic_Params_Tuning_XGBoost.csv')

# Decide optimum set of parameters for XGBoost algorithm

In [ ]:
# See if any values larger than 1 in column "Product"
# If yes, the corresponding set of parameters occur more than once and those sets should be considered first
df_output_hyper_params.groupby(['subsample', 'n_estimators', 'max_depth', 
                         'learning_rate', 'colsample_bytree',
                         'colsample_bylevel'])[['Product']].count().sort_values(by = 'Product', ascending=False)

Product
subsample n_estimators max_depth learning_rate colsample_bytree colsample_bylevel         
0.8       30.0         10.0      0.01          0.9              0.5                      3
0.6       30.0         10.0      0.01          0.9              0.7                      3
                                               0.6              0.7                      2
0.8       30.0         7.0       0.01          0.9              0.5                      2
0.7       30.0         10.0      0.01          0.9              0.8                      2
...                                                                                    ...
                       7.0       0.01          0.8              0.5                      1
                                                                0.9                      1
                       10.0      0.01          0.5              0.8                      1
                                                                0.9                      1
0.9       100.0        5.0       0.20          0.6              0.6                      1

[69 rows x 1 columns]

In [ ]:
# See what value of parameter "subsample" occurs the most frequently
df_output_hyper_params.groupby(['subsample'])[['Product']].count()

,Product
subsample,
0.5,14
0.6,20
0.7,11
0.8,20
0.9,15


In [ ]:
# See what value of parameter "n_estimators" occurs the most frequently
df_output_hyper_params.groupby(['n_estimators'])[['Product']].count()

,Product
n_estimators,
30.0,73
50.0,3
100.0,4


In [ ]:
# See what value of parameter "max_depth" occurs the most frequently
df_output_hyper_params.groupby(['max_depth'])[['Product']].count()

,Product
max_depth,
3.0,3
5.0,9
7.0,25
10.0,43


In [ ]:
# See what value of parameter "learning_rate" occurs the most frequently
df_output_hyper_params.groupby(['learning_rate'])[['Product']].count()

,Product
learning_rate,
0.01,72
0.10,2
0.20,4
0.30,2


In [ ]:
# See what value of parameter "colsample_bytree" occurs the most frequently
df_output_hyper_params.groupby(['colsample_bytree'])[['Product']].count()

,Product
colsample_bytree,
0.5,10
0.6,12
0.7,15
0.8,15
0.9,28


In [ ]:
# See what value of parameter "colsample_bytree" occurs the most frequently
df_output_hyper_params.groupby(['colsample_bylevel'])[['Product']].count()

,Product
colsample_bylevel,
0.5,17
0.6,12
0.7,17
0.8,17
0.9,17


# Implement XGBoost on Test Set with optimum set of parameters

### Specify optimum set of parameters

In [22]:
subsample_chosen = 0.8
n_estimators_chosen = 30
max_depth_chosen = 10
learning_rate_chosen = 0.1               
colsample_bytree_chosen =  0.9
colsample_bylevel_chosen = 0.5

In [23]:
len(list_pattern_py)

99

In [24]:
list_pattern_temp = list_pattern_py.copy()

In [25]:
start = timer()

df_output_xgb_pattern = pd.DataFrame()
for product in list_pattern_temp:
  print('Currently Running: %s' % product)
  print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))
  dataframe_product = dataframe_weekly[['wm_yr_wk', product]]
  # Create Lag variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (t-{lag})': dataframe_product[product].shift(lag)
      for lag in lags
  })

  # Create Rolling Moving Average variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
      for moving_average in moving_averages
  })

  # Create Rolling Standard Deviation variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
      for std in stds
  })

  # Create Rolling Maximum variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
      for maximum in maximums
  })

  # Create Rolling Minimum variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
      for minimum in minimums
  })

  # Create Rolling Total variables 
  dataframe_product = dataframe_product.assign(**{
      f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
      for total in totals
  })

  dataframe_product.dropna(inplace=True)
  dataframe_product.set_index('wm_yr_wk', inplace=True)

  train_data = dataframe_product[dataframe_product.index <= end_train_week]
  test_data = dataframe_product[dataframe_product.index > end_train_week]

  corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
  high_corr_var=np.where(corr_matrix > correlation_threshold)
  high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

  train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
  test_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)

  X_train = train_data.drop(product, axis=1)
  y_train = train_data[product]

  X_test = test_data.drop(product, axis=1)
  y_test = test_data[product]

  xgb_r = xgb.XGBRegressor(seed = 20,
                          subsample = subsample_chosen,
                          n_estimators = n_estimators_chosen,
                          max_depth = max_depth_chosen,
                          learning_rate = learning_rate_chosen,               
                          colsample_bytree = colsample_bytree_chosen,
                          colsample_bylevel = colsample_bylevel_chosen)
  

  xgb_r.fit(X_train,y_train)
  y_pred = xgb_r.predict(X_test)
  df_output_temp = pd.DataFrame({'Actual Data': y_test, 
                                  'Forecast': y_pred, 
                                  'Product': [product for num_rows in range(len(y_pred))]
                                  })
  
  df_output_xgb_pattern = df_output_xgb_pattern.append(df_output_temp, ignore_index=False)  

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: HOBBIES_1_002_CA_1
Progressing: 0.0 %
[03:57:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running: HOBBIES_1_003_CA_1
Progressing: 1.01 %
[03:57:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running: HOBBIES_1_076_CA_1
Progressing: 2.02 %
[03:57:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running: HOBBIES_1_080_CA_1
Progressing: 3.03 %
[03:57:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running: HOBBIES_1_091_CA_1
Progressing: 4.04 %
[03:57:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running: HOBBIES_1_111_CA_1
Progressing: 5.05 %
[03:57:36] WARNING

In [26]:
df_output_xgb_pattern.to_csv('Erratic_XGBoost_Forecast_Values.csv')

In [27]:
# User-defined funcion to calculate metrics

def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [28]:
# Start calculating metrics 
start = timer()

df_forecast_metrics = df_output_xgb_pattern.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.008508452299999665 minutes


In [29]:
df_forecast_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_002_CA_1,1.132744,226.5489,128.0492,77.9678,1.132744,1.186493
1,HOBBIES_1_003_CA_1,inf,inf,200.0000,NaN,0.021140,0.021140
2,HOBBIES_1_008_CA_3,0.720427,62.5825,56.3854,97.6390,30.978354,36.319828
3,HOBBIES_1_009_CA_2,2.031118,23.8955,27.0057,23.7651,2.031118,2.054228
4,HOBBIES_1_032_CA_3,0.670333,24.1760,25.6568,20.7970,7.373668,10.070547
...,...,...,...,...,...,...,...
94,HOBBIES_2_129_CA_2,2.211799,44.2360,56.3368,43.8690,4.423599,4.496367
95,HOBBIES_2_132_CA_2,inf,inf,200.0000,NaN,0.026744,0.026744
96,HOBBIES_2_142_CA_2,1.571856,56.4256,77.7248,55.9167,11.002992,11.285636
97,HOBBIES_2_142_CA_3,0.579700,92.1574,104.1499,247.0322,17.970695,17.973127


In [30]:
df_forecast_metrics.to_csv('XGBoost_Erratic_Forecast_Metrics.csv')